In [56]:
import pandas as pd
import numpy as np
import geopandas as gpd
import timeit
import scipy

from scipy import stats
from pandas import ExcelWriter

pd.set_option('display.max_columns', None)
idx = pd.IndexSlice

In [57]:
input_path = '../../data/input/dhis2/new_system/'
input_path_old = '../../data/input/dhis2/old_system/'

shapes_path = '../../data/shapes/district/districts_17_19_clean.shp'
facility_path = '../../data/input/hospitals/original_data/'

output_path = '../../data/output/sprint3_analysis/'

# Fetch my data


## Set up

In [58]:
# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [59]:
# Creating a dict of names to replace district names

district_name_dict = {'SEMBABULE': 'SSEMBABULE', 'MADI-OKOLLO': 'MADI OKOLLO'}

# For a fully automated one, will need to do fuzzy matching


In [60]:
# To clean the data downloaded in a pivot format

def get_clean_stack(df,drop):

    df['district']=df['orgunitlevel3'].apply(lambda x: x[:-9].upper())
    df['district'].replace(district_name_dict,inplace=True)
    
    df.set_index(['district','organisationunitid'],drop=True,inplace=True)
    
    cols = np.arange(0,9)
    df.drop(df.columns[cols],axis=1,inplace=True)
    
    cols = df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols,names=['year','indic','month'])
    
    if drop != None:
        df.drop(drop,axis=1,inplace=True,level=2)
    
    df1=df.copy().stack(level=[0,1,2],dropna=False).reset_index()
    df1.rename(columns={0:'value'},inplace=True)
    
    return df1

## New data


In [61]:
# to get all the new data files together
def fetch_new_data (filepaths):
    df = pd.DataFrame(columns = ['district', 'organisationunitid', 'year', 'indic', 'month', 'value'])
    for x in filepaths:
        x_df=get_clean_stack(pd.read_csv(x),drop='Dec')
        df=pd.concat([df,x_df])
    for x in filepaths_no_dec:
        x_df=get_clean_stack(pd.read_csv(x),drop=None)
        df=pd.concat([df,x_df])
    return df


In [62]:
# creating the new data stack

filepaths = [input_path+'new_epi_data_by_facility.csv',
            input_path+'new_mnch_data_by_facility.csv',
            input_path+'new_sam_data_by_facility.csv',
            input_path+'new_lbw_data_by_facility.csv',
            input_path+'new_vitamin_data_by_facility.csv']

filepaths_no_dec = [input_path+'may_new_sam_data_by_facility.csv',
                    input_path+'may_new_epi_data_by_facility.csv']

new_stack = fetch_new_data (filepaths=filepaths)

## Old data

For now extremely messy, had to be done bit by bit in random order, so quite some cleaning needed Ill do here, than put all into one nice file

In [63]:
# to get all the old data files together
def fetch_old_data (filepaths):
    df = pd.DataFrame(columns = ['district', 'organisationunitid', 'year', 'indic', 'month', 'value'])
    for x in filepaths:
        x_df=get_clean_stack(pd.read_csv(x),drop=None)
        df=pd.concat([df,x_df])
    return df


In [64]:

# creating the old data stack

# For now a bit all over the placedue to download limitations 

filepaths = [input_path_old+"/epi/EPI - BCG doses given.csv",
             input_path_old+"/epi/EPI - DPT-HepB-HIB 1 doses given.csv",
             input_path_old+"/epi/EPI - DPT-HepB-HIB 3 doses given.csv",
             input_path_old+"/epi/EPI - PCV 1 doses given.csv",
             input_path_old+"/epi/EPI - PCV 3 doses given.csv",
             input_path_old+"/epi/EPI - MR 1 doses given.csv",
             input_path_old+'/mat/admission_newborn.csv',
             input_path_old+'/mat/ANC1_ANC4.csv',
             input_path_old+'/mat/births.csv',
             input_path_old+'/sam/lbw.csv',
             input_path_old+'/sam/sam_mam.csv',
             input_path_old+'/sam/lbw_abs.csv',
             input_path_old+'/sam/vitamin.csv']

old_stack = fetch_old_data (filepaths=filepaths)

# Merge the two

In [66]:
# Bridging differences between lbw variables in old and new system

#lbw_elements = ['105-MA04b2. Deliveries in unit -Live births - less than 2.5kg',
                #'105-MA04c2. Deliveries in unit - Fresh still birth - less than 2.5kg',
                #'105-MA04d2. Deliveries in unit - Macerated still birth - less than 2.5kg']

#lbw_data = new_stack[new_stack['indic'].isin(lbw_elements)].groupby(['district', 'organisationunitid', 'year', 'month'],as_index=False).sum()
#lbw_data['indic']='Babies Born with low birth weight (<2.5Kgs)'
#lbw_data

## Select a subset of indicators to merge on

This is done in Excel manually, after running the small function below

In [67]:
def export_var_names():
    from pandas import ExcelWriter
    with ExcelWriter(input_path+'new_old_correspondance.xlsx',mode='a') as writer:
        pd.Series(old_stack['indic'].unique()).to_excel(writer,sheet_name='old_vars')
        pd.Series(new_stack['indic'].unique()).to_excel(writer,sheet_name='new_vars')

#export_var_names()

Here I get back the result into a dict

In [68]:
def replace_var_names():
    from pandas import ExcelFile
    xls = ExcelFile(input_path+'new_old_correspondance.xlsx')
    df = xls.parse(xls.sheet_names[0])
    df.set_index('Old',drop=True,inplace=True)
    old_new_dict=df['New'].to_dict()
    old_stack.replace({'indic': old_new_dict},inplace=True) # Replacing the old names by the new
    target_indics=list(old_new_dict.values()) # Store my target indicators
    return target_indics

In [69]:
target_indics=replace_var_names()

In [70]:
target_indics

['105-AN01a. ANC 1st Visit for women',
 '105-AN02. ANC 4th Visit for women',
 '105-MA01. Admissions',
 '105-MA04a. Deliveries in unit - Total',
 '105-MA04b1. Deliveries in unit -Live births - Total',
 '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
 '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
 '105-MA11. Newborn deaths (0-7 days)',
 '105-PN01a. Post Natal Attendances - Mother',
 'EPI - BCG doses given',
 'EPI - DPT-HepB-HIB 1 doses given',
 'EPI - DPT-HepB-HIB 3 doses given',
 'EPI - MR 1 doses given',
 'EPI - PCV 1 doses_Under 1',
 'EPI - PCV 3 doses_Under 1',
 'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
 '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
 '105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema',
 '105-NA03c1. Identified malnourished clients(<10) this month - SAM using MUAC -  Without Oedema',
 '105-CH01. Vit A supplement (1st Dose)',
 '105-CH02. Vit A

### Check there is no issue with facility ids

In [71]:
# Store the valid ids

old_ids=set(old_stack['organisationunitid'].unique())
new_ids=set(new_stack['organisationunitid'].unique())
valid_ids=list(old_ids.intersection(new_ids))


### Get only the bits of data I am interested in

In [72]:
old_stack_t=old_stack[old_stack['indic'].isin(target_indics) & old_stack['organisationunitid'].isin(valid_ids)].copy()
new_stack_t=new_stack[new_stack['indic'].isin(target_indics) & new_stack['organisationunitid'].isin(valid_ids)].copy()

In [73]:
stack_t=pd.concat([old_stack_t,new_stack_t])
stack_t.reset_index(drop=True,inplace=True)
stack_t.head()

,district,organisationunitid,year,indic,month,value
0,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Apr,7.0
1,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Aug,7.0
2,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Dec,5.0
3,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Feb,NaN
4,ZOMBO,XikHv88zzDn,2018,EPI - BCG doses given,Jan,NaN


In [74]:
stack_t.shape

(3463896, 6)

In [75]:
#stack_t.loc[3076856]
#stack_t.loc[3068237]
#stack_t.loc[3068236]
#stack_t.loc[2495098]

# Flag outliers

## (!) Directly replace data that is out of bound (for nutrition)

In [76]:
lbw_only = stack_t[stack_t['indic']=='NUT: Percentage of children/babies born with low birth weight (<2.5kg)']

for x in list(lbw_only[lbw_only['value']>100].index):
    stack_t.loc[x,'value']=np.nan


# Put our data in the right format 


In [77]:
%%time

def pivot_stack(df):
    pivot_outliers=df.copy().pivot_table(index=['district', 'organisationunitid', 'indic'], columns=['year','month' ]) #,dropna=False)
    pivot_outliers.rename(columns={'value':'with_outiers'},level=0,inplace=True)
    pivot_outliers.columns.rename('type', level=0, inplace=True)
    pivot_outliers.dropna(how='all',axis=0,inplace=True) # looks like there is no all na line to drop
    return pivot_outliers

pivot_outliers=pivot_stack(stack_t)

Wall time: 5.23 s


## Replace outliers using a std deviation method

In [78]:
# Get the data in the right format

def replace_outliers(pivot_outliers,cutoff):#df
    
    pivot_no_outliers=pd.DataFrame(columns=pivot_outliers.columns,index=pivot_outliers.index)
    pivot_no_outliers.rename(columns={'with_outiers':'without_outliers'},level=0,inplace=True)
    
    for x in pivot_outliers.index: # to exclude
        values = pivot_outliers.loc[x,:].values
        if np.nanstd(values)!=0 and np.isnan(values).sum()!=len(values):
            zscore = abs(stats.zscore(values,nan_policy='omit'))
            new_values = np.where(zscore>cutoff,np.nanmedian(values),values)

        else:
            new_values = values

        pivot_no_outliers.iloc[pivot_outliers.index.get_loc(x),:] = new_values.astype('float')

    return pivot_no_outliers 


In [79]:
%%time
pivot_no_outliers = replace_outliers(pivot_outliers,cutoff=3)


<ipython-input-78-34eb04742e5b>:12: RuntimeWarning: invalid value encountered in greater
  new_values = np.where(zscore>cutoff,np.nanmedian(values),values)


Wall time: 2min 6s


In [80]:
values = np.array([1,3,np.nan,3,1,4,6,4,1000,200])
Q1 = np.nanquantile(values,0.25)
Q3 = np.nanquantile(values,0.75)
IQR = Q3 - Q1
LB = Q1 - 3*IQR
UB = Q3 + 3*IQR
np.where((values<LB)|(values>UB),np.nanmedian(values),values)

<ipython-input-80-0516574b05c8>:7: RuntimeWarning: invalid value encountered in less
  np.where((values<LB)|(values>UB),np.nanmedian(values),values)
<ipython-input-80-0516574b05c8>:7: RuntimeWarning: invalid value encountered in greater
  np.where((values<LB)|(values>UB),np.nanmedian(values),values)


array([ 1.,  3., nan,  3.,  1.,  4.,  6.,  4.,  4.,  4.])

In [81]:
# Get the data in the right format

def replace_outliers_iqr(pivot_outliers,k):#df
    
    pivot_no_outliers=pd.DataFrame(columns=pivot_outliers.columns,index=pivot_outliers.index)
    pivot_no_outliers.rename(columns={'with_outiers':'without_outliers'},level=0,inplace=True)
    
    for x in pivot_outliers.index:
        values = pivot_outliers.loc[x,:].values
        if np.nanstd(values)!=0 and np.isnan(values).sum()!=len(values):
            Q1 = np.nanquantile(values,0.25)
            Q3 = np.nanquantile(values,0.75)
            IQR = Q3 - Q1
            LB = Q1 - k*IQR
            UB = Q3 + k*IQR
            new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)

        else:
            new_values = values

        pivot_no_outliers.iloc[pivot_outliers.index.get_loc(x),:] = new_values.astype('float')

    return pivot_no_outliers 

In [82]:
%%time
pivot_no_outliers_iqr = replace_outliers_iqr(pivot_outliers,k=3)

<ipython-input-81-53464bcc185b>:16: RuntimeWarning: invalid value encountered in less
  new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)
<ipython-input-81-53464bcc185b>:16: RuntimeWarning: invalid value encountered in greater
  new_values = np.where((values<LB)|(values>UB),np.nanmedian(values),values)


Wall time: 2min 36s


In [83]:
pivot_no_outliers_iqr.head()

type                                                              without_outliers  \
year                                                                          2018   
month                                                                          Apr   
district organisationunitid indic                                                    
ABIM     JO1cLIghdBv        105-AN01a. ANC 1st Visit for women                  17   
                            105-AN02. ANC 4th Visit for women                    5   
                            105-CH01. Vit A supplement (1st Dose)               60   
                            105-CH02. Vit A supplement (2nd Dose)               93   
                            105-MA01. Admissions                                 7   

type                                                                           \
year                                                                            
month                                                             Aug Dec Feb   
district organisationunitid indic                                               
ABIM     JO1cLIghdBv        105-AN01a. ANC 1st Visit for women     21   6   8   
                            105-AN02. ANC 4th Visit for women       6   3   4   
                            105-CH01. Vit A supplement (1st Dose)  91  68   9   
                            105-CH02. Vit A supplement (2nd Dose)  22  40   5   
                            105-MA01. Admissions                   14   4   4   

type                                                                        \
year                                                                         
month                                                              Jan Jul   
district organisationunitid indic                                            
ABIM     JO1cLIghdBv        105-AN01a. ANC 1st Visit for women     NaN   9   
                            105-AN02. ANC 4th Visit for women        6  10   
                            105-CH01. Vit A supplement (1st Dose)  NaN  14   
                            105-CH02. Vit A supplement (2nd Dose)    3   3   
                            105-MA01. Admissions                   NaN   9   

type                                                                           \
year                                                                            
month                                                             Jun Mar May   
district organisationunitid indic                                               
ABIM     JO1cLIghdBv        105-AN01a. ANC 1st Visit for women      8  19  10   
                            105-AN02. ANC 4th Visit for women       4   7   4   
                            105-CH01. Vit A supplement (1st Dose)   3   8   9   
                            105-CH02. Vit A supplement (2nd Dose)   1  13  10   
                            105-MA01. Admissions                   10   8   3   

type                                                                         \
year                                                                          
month                                                              Nov  Oct   
district organisationunitid indic                                             
ABIM     JO1cLIghdBv        105-AN01a. ANC 1st Visit for women       4   12   
                            105-AN02. ANC 4th Visit for women        3   11   
                            105-CH01. Vit A supplement (1st Dose)   31  158   
                            105-CH02. Vit A supplement (2nd Dose)  190  192   
                            105-MA01. Admissions                     2   11   

type                                                                        \
year                                                                  2019   
month                                                             Sep  Apr   
district organisationunitid indic                                            
ABIM     JO1cLIghdBv        105-AN01a.

## Stack the outlier corrected data

In [84]:
def pivot_stack(pivot):
    stack = pivot.stack(level=[0,1,2],dropna=False).reset_index()
    stack.rename(columns={0:'value'},inplace=True)
    stack.drop('type',axis=1,inplace=True)
    stack['value']=stack['value'].astype(dtype='float64')
    return stack



In [85]:
stack_t_noout=pivot_stack(pivot_no_outliers)
stack_t_noout_iqr=pivot_stack(pivot_no_outliers_iqr)


## Record which data points were changed

In [86]:
stack_compare = pd.merge(stack_t,stack_t_noout,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})
stack_compare.dropna(subset=['value_out','value_noout'],inplace=True)
stack_compare['changed']=np.where((stack_compare['value_out'] != stack_compare['value_noout']),True,False)
changed = stack_compare[stack_compare['changed']==True]

In [87]:
stack_compare_iqr = pd.merge(stack_t,stack_t_noout_iqr,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})
stack_compare_iqr.dropna(subset=['value_out','value_noout'],inplace=True)
stack_compare_iqr['changed']=np.where((stack_compare_iqr['value_out'] != stack_compare_iqr['value_noout']),True,False)
changed_iqr = stack_compare_iqr[stack_compare_iqr['changed']==True]

In [88]:
changed_iqr

,district,organisationunitid,year,indic,month,value_out,value_noout,changed
41,LUWERO,T4M9UgfqV5q,2019,EPI - BCG doses given,Jul,44.0,22.0,True
394,PADER,auzuV39xOTU,2018,EPI - BCG doses given,Oct,58.0,12.0,True
831,AMURIA,o7CJeTwDapk,2019,EPI - BCG doses given,Feb,20.0,4.0,True
928,MOYO,ZubUtSX9erU,2019,EPI - BCG doses given,Jan,6.0,2.0,True
1010,ABIM,ltIECAx2ppI,2018,EPI - BCG doses given,Dec,57.0,7.0,True
...,...,...,...,...,...,...,...,...
2343762,BUYENDE,DCjQKtUiiHH,2020,EPI - DPT-HepB-HIB 3 doses given,May,185.0,60.5,True
2343763,BUYENDE,DCjQKtUiiHH,2020,EPI - MR 1 doses given,May,176.0,52.0,True
2343764,BUYENDE,DCjQKtUiiHH,2020,EPI - PCV 1 doses_Under 1,May,214.0,61.5,True
2343765,BUYENDE,DCjQKtUiiHH,2020,EPI - PCV 3 doses_Under 1,May,177.0,54.0,True


In [89]:
changed.to_csv(output_path+'outliers_list.csv')
changed_iqr.to_csv(output_path+'outliers_list_iqr.csv')

# Export this to Tableau

In [90]:
# to check any data point below:

#stack_t[(stack_t['organisationunitid']=='JO1cLIghdBv') & 
        #(stack_t['year']=='2018') & 
        #(stack_t['indic']=='105-NA03e1. Identified malnourished clients(<10) this month - SAM With Oedema') & 
        #(stack_t['month']=='Apr')]['value'].notna()

In [98]:
fac_stack_final = pd.merge(stack_t,stack_t_noout,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value_x':'value_out','value_y':'value_noout'})

fac_stack_final = pd.merge(fac_stack_final,stack_t_noout_iqr,how='inner',
                           left_on=['district', 'organisationunitid', 'year', 'indic', 'month'],
                           right_on=['district', 'organisationunitid', 'year', 'indic', 'month']).rename(columns={'value':'value_noout_iqr'})

fac_stack_final['reported'] = fac_stack_final['value_out'].notna().astype('int')

fac_pivot_final=fac_stack_final.pivot_table(index=['district','organisationunitid','year','month'], columns=['indic'])

# The pivot creates new NaN values, that need to bereplaced by False for the 'reported' metric

for i in list(fac_pivot_final.columns.levels[1]):
    fac_pivot_final[('reported',i)].replace({np.nan:0},inplace=True)

fac_pivot_final=fac_pivot_final.stack(level=[0])

In [100]:
pivot_export=fac_pivot_final.copy()
pivot_export.to_csv(output_path+'corrected_data_facility.csv')

In [101]:
pivot_export.shape

(522053, 21)